# MuMoT User Manual
## Multiscale Modelling Tool

This is the user manual for [MuMoT](https://github.com/DiODeProject/MuMoT), a software tool developed at the University of Sheffield as part of the [DiODe](http://diode.group.shef.ac.uk) project

*Current MuMoT version*: **0.0**

## Working with MuMoT

MuMoT runs inside [Jupyter notebooks](http://jupyter.org) - since you are reading this User Manual you have presumably already installed Jupyter, or are using an installation provided to you

Next, you need to install MuMoT itself; since MuMoT is a Python package it can be installed from PyPi - see the [project page](https://diodeproject.github.io/MuMoT/) for more details

To run commands in 'Code' cells it's usually easiest just to hit `Shift + Enter`; here is some basic information on [working with Jupyter notebooks](http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html)

With Jupyter and MuMot both installed, you simply need to import the MuMoT package into your notebook; we recommend you refer to it as `mmt`

In [ ]:
import MuMoT.MuMoT as mmt

## Defining your first model

If everything went well with importing MuMoT you should have seen the message

``Created `%%model` as an alias for `%%latex``

MuMoT models are defined using the `%%model` keyword within a cell, and have a very simple syntax; since MuMoT works with *chemical kinetic* or *reaction kinetic* models, you simply have to describe the reactions that take place, and the rates with which they occur. For example

`A + A -> A + U: s`

is a very simple reaction, in which two particles of type `A` interact, and one changes to type `U`, at rate `s`

These kind of reaction rules are quite general; they describe a lot of chemical reactions, as well as collective behaviour in which individuals interact to change each other's state, and even demographic models in which individuals are born and die

Our first model will be based on signalling behaviour observed in honeybee swarms, as described in the following papers:
* Seeley, T.D, Visscher, P.K. Schlegel, T., Hogan, P.M., Franks, N.R. & Marshall, J.A.R. (2012) [Stop signals provide cross inhibition in collective decision-making by honeybee swarms](http://www.sciencemag.org/content/335/6064/108.full.pdf). *Science* **335**, 108-111
* Pais, D., Hogan, P.M., Schlegel, T., Franks, N.R., Leonard, N.E. & Marshall, J.A.R. (2013) [A mechanism for value-sensitive decision-making](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0073216).  *PLoS one* **8**(9), e73216

The `$` at the start and end just allow us to use LaTeX [codes](http://reu.dimacs.rutgers.edu/Symbols.pdf) for our state and rate labels, allowing us to use  letters from different alphabets, such as Greek, and other nice formatting - if you don't know LaTeX don't worry about this - leave in the `$` signs, they won't hurt

**N.B.** currently it is important to put spaces between all elements of each rule, as below

OK, here we go...

In [ ]:
%%model
$
U -> A : g_A
U -> B : g_B
A -> U : a_A
B -> U : a_B
A + U -> A + A : r_A
B + U -> B + B : r_B
A + B -> A + U : s
A + B -> B + U : s
$

Now we have our model, and Jupyter should have reprinted the rules underneath the cell you input them in

You can see that actually the `_` code enables you to use subscripts, which can be handy for distinguishing related rates - in this model there are two distinct nest sites for the honeybee swarm to choose between, *A* and *B*, and rates associated with each one

We have our model, but currently it is not useful; it's only a definition, not a fully-fledged model that we can start analysing with MuMoT. To make one of those we have to use the `parseModel` command; we simply pass in a reference to the cell where we defined our model...

In [ ]:
model1 = mmt.parseModel(In[2])

If there is no output from `parseModel` that is good news - it means the model was valid. Now we have an object, which we called `model1`, which is a version of the model that we can start doing useful work with. For example, we can see its 'reactants' (or states, or types of individual; remember the terminology comes from chemical reactions)...

In [ ]:
model1.showReactants()

...and the rates the reactions happen at

In [ ]:
model1.showRates()

Note that here MuMoT also helpfully tells us which reactions in the original model definition each rate is associated with

We can also look at the reaction equations in a slightly nicer format...

In [ ]:
model1.show()

We can even see a simple figure representing our model...

In [ ]:
model1.visualise()

This figure attempts to represent the nature of the interactions between reactants graphically, from simple transitions (arrows), inhibitions (filled circles) and induced switches (arrows with filled circles at the origin); to understand the different interaction patterns try relating the arrows with the reactions we defined above, by reference to the rates they are labelled with

## Exploring your new model

If you want to know what commands you can send to an object, Jupyter lets you type out the object name, then a `.`, then hit `Tab` to see a list of accepted commands

One commmand that looks intruiguing is `showODEs`

In [ ]:
model1.showODEs()

This is where things start getting interesting, because equations are what mathematicians analyse. `showODEs` shows the model's Ordinary Differential Equations, which describe how the populations of reactants in the model change over time. MuMoT has derived these equations for you automatically, from your description of the reactions. If you are familiar with ODEs then you should be able to read these quite easily. However even if you are not, you can still work with them

So, what analysis can we do in MuMoT now we've found our ODEs?

Well, we might want to look at how the state of the system (the proportions of the different reactants) change on average over time...

...but before we can do that, we need to let MuMoT know that this is actually a simpler set of equations than it appears to be. $A$, $B$, and $U$ represent states that honebees can be in during the swarming process, either committed to site *A*, committed to site *B*, or *U*ncommitted. Because there is a constant number of individuals in the swarm during a decision, one of these equations is redundant; the change in the uncommitted bee frequency can be worked out from the change in frequencies of A-committed and B-committed bees, for example

We let MuMoT know this by defining one of the reactants in terms of the other, using `substitute`

In [ ]:
model2 = model1.substitute('U = N - A - B')

Now we have a simplified model, `model2`, which only has two equations, because we have told MuMoT that the total number of uncommitted bees $U$ is the total number of bees in the swarm $N$, minus the numbers committed to the two different nests ($A$ and $B$)

In [ ]:
model2.showODEs()

Now we can look at how the model behaves, by asking for a vector plot of the two reactants $A$ and $B$...

In [ ]:
vector1 = model2.vector('A', 'B')

Now we can see how we can start working with models in MuMoT. The figure you see is a vector plot, which shows on average how the state of the system (proportions of bees in states $A$ and $B$) will change over time, from any point in the state space. The arrows give the direction the system will move in, and their lengths show how fast. Because the total number of bees is constant, $A+B$ can never exceed 1, so the top right hand triangle of the figure is greyed out; these are impossible states. The number of uncommitted bees $U$, which we hid away earlier, is not shown, but it is simply $1-A-B$. But what we really care about in a decision-making process is how the number of 'voters' for the competing options changes over time

You will also notice that there are some sliders above the figure; MuMoT gives you a slider for every rate in the model, and by changing their values, you can see what happens to the behaviour of the system

Jupyter also lets you interact nicely with the figure; you can zoom in, back out, pan around, and save, all by clicking the icons that appear underneath it. If you want to save a figure for a paper, simply click the disk icon. And if while exploring the model you find a nice figure, you can click the 'off' icon in its top right to snapshot it, and generate a new interactive figure to play with. You can then simply right-click your snapshots to save them from your browser - just don't forget to note the parameters that generated them - you can find that out by looking at the logs...

Every 'controller' in MuMoT keeps a log of everything it's been asked to do; we called our controller `vector1` when we created it, so it is easy to check what it has been up to... (WARNING: if you have been playing with the sliders a lot, there could be a lot in the logs!)

In [ ]:
vector1.showLogs()

For papers and presentations we can produce somewhat prettier plots similar to figure 1 above, using `stream`. But before we do that, one unfortunate thing about our `vector1` controller is that there are just too many sliders. We might choose to describe all the rates (*a*bandonment, *r*ecruitment (via the waggle dance), and individual discovery and commitment (*g*)) in terms of the *v*alues of the nest sites... we can do this by using `substitute` again

In [ ]:
model3 = model2.substitute('a_A = 1/v_A, a_B = 1/v_B, g_A = v_A, g_B = v_B, r_A = v_A, r_B = v_B')

Our new model is a little simpler, with just three rates, $v_A$, $v_B$ and $s$

In [ ]:
model3.show()

Still, in decision-making what we really care about is how good options are on average, and their difference in value. Pais et al. (2013) thus defined option values in terms of deviation from the average value, so
$$\displaystyle v_A=\mu+\frac{\Delta}{2}$$
and
$$\displaystyle v_B=\mu-\frac{\Delta}{2}$$

In [ ]:
model4 = model3.substitute('v_A = \\mu + \\Delta/2, v_B = \\mu - \\Delta/2')
model4.showODEs()

This model is much easier to interact with...

In [ ]:
stream1 = model4.stream('A','B', showFixedPoints = True)

A `stream` plot is just like a `vector` plot, except now lines show the average change of the system over time in more detail, and their shading represents the speed of change, from slow (light grey) to fast (black)

Note also that here we added a *keyword argument* `showFixedPoints = True` - this can also be applied to `vector` plots, and plots the stationary points of the equation. If the stationary point is *stable* (arrows converge on that point) then it is represented with a filled circle, and if it is *unstable* (arrows leave that point) then a hollow circle is shown. Displaying fixed points can slow down plots, so by default they are not shown, and you need to explicitly request they are included